In [ ]:
import threading
import csv
from statistics import mean, median, stdev

class FileProcessor:
    def __init__(self):
        self.lock = threading.Lock()
        self.results = {}

    def process_files(self, file_names):
        threads = []
        for file_name in file_names:
            thread = threading.Thread(target=self.process_file, args=(file_name,))
            threads.append(thread)
            thread.start()

        for thread in threads:
            thread.join()

    def process_file(self, file_name):
        try:
            with open(file_name, 'r') as file:
                reader = csv.reader(file)
                data = list(reader)

                # Assuming the first row is headers
                headers = data[0]
                numerical_data = {header: [] for header in headers if self.is_number(data[1][headers.index(header)])}

                for row in data[1:]:
                    for header in numerical_data:
                        value = row[headers.index(header)]
                        if self.is_number(value):
                            numerical_data[header].append(float(value))

                statistics = {header: self.calculate_statistics(values) for header, values in numerical_data.items()}

                with self.lock:
                    self.results[file_name] = statistics

        except Exception as e:
            with self.lock:
                self.results[file_name] = {"error": str(e)}

    def get_results(self):
        return self.results

    @staticmethod
    def is_number(s):
        try:
            float(s)
            return True
        except ValueError:
            return False

    @staticmethod
    def calculate_statistics(values):
        if values:
            return {
                "mean": mean(values),
                "median": median(values),
                "std_dev": stdev(values) if len(values) > 1 else 0
            }
        else:
            return {"mean": None, "median": None, "std_dev": None}

# Example usage
file_processor = FileProcessor()
file_processor.process_files(["file1.csv", "file2.csv"])
results = file_processor.get_results()
results

